In [5]:
import pytesseract
import numpy as np
from tqdm.notebook import tqdm
import os
import pandas as pd


In [6]:
print(pytesseract.get_tesseract_version())
os.environ["TESSDATA_PREFIX"] = "/Users/christopherczaban/Local_Project_Files/OCR_Tests/tessdata"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

5.3.4


In [7]:
from transformers.processing_utils import ProcessorMixin
from transformers.tokenization_utils_base import BatchEncoding, PaddingStrategy, PreTokenizedInput, TextInput, TruncationStrategy
from transformers.utils import TensorType

/Users/christopherczaban/Local_Project_Files/HuggingFace_Models/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from transformers import LiltForSequenceClassification
from datasets import load_dataset, load_dataset_builder
from PIL import Image, ImageDraw, ImageFont, ImageFilter
from typing import List,Dict, Iterable, Optional, Union
import torch
from torch.utils.data import Dataset

In [ ]:
torch.has_mps

In [ ]:
!huggingface-cli login

# Getting the data

In [ ]:
from datasets import disable_caching

disable_caching()

In [16]:
data_directory = {
    "base_dir": "/Users/christopherczaban/Local_Project_Files/Kundenprojekte_Code_lokal/Code_pma/Dokumente_Kategorisierung/Dokumente/PoV-Kategorie-Split_Erste_und_Weitere-Seiten-gemischt",
    "train": "/Users/christopherczaban/Local_Project_Files/Kundenprojekte_Code_lokal/Code_pma/Dokumente_Kategorisierung/Dokumente/PoV-Kategorie-Split_Erste_und_Weitere-Seiten-gemischt/train",
    "test": "/Users/christopherczaban/Local_Project_Files/Kundenprojekte_Code_lokal/Code_pma/Dokumente_Kategorisierung/Dokumente/PoV-Kategorie-Split_Erste_und_Weitere-Seiten-gemischt/test"
    }
data_directory_train = data_directory['train']
data_directory_test = data_directory['test']

In [18]:
import os

from datasets import load_dataset
from datasets import Dataset as hfds
from datasets import DatasetDict

def load_imagefolder(data_dir):
    image_folder_dataset = load_dataset("imagefolder", data_dir)

    images = [os.path.join(data_dir, path) for path, _ in image_folder_dataset.imgs]
    labels = image_folder_dataset.targets

    image_data = []
    for image_path in images:
        image = Image.open(image_path)
        image_data.append(image)

    data = {"image": image_data, "image_path": images, "label": labels}

    return hfds.from_dict(data)

ModuleNotFoundError: No module named 'torchvision'

In [534]:
dataset_train = load_imagefolder(data_directory_train)
dataset_train

Dataset({
    features: ['image', 'image_path', 'label'],
    num_rows: 85
})

In [535]:
dataset_test = load_imagefolder(data_directory_test)
dataset_test

Dataset({
    features: ['image', 'image_path', 'label'],
    num_rows: 17
})

In [536]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test})

In [537]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'image_path', 'label'],
        num_rows: 85
    })
    test: Dataset({
        features: ['image', 'image_path', 'label'],
        num_rows: 17
    })
})

In [ ]:
#dataset = DatasetDict({"test": dataset_test})

In [ ]:
dataset = load_dataset("imagefolder", data_dir=data_directory, drop_labels=False)

In [498]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 85
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 17
    })
})

In [ ]:
#dataset['train']['image'][10]

In [ ]:
def show_image(path):
    return Image.open(path).convert("RGB")

In [ ]:
# Test image
path_to_img = "/Users/christopherczaban/Local_Project_Files/Kundenprojekte_Code_lokal/Code_pma/Dokumente_Kategorisierung/Dokumente/PoV-Kategorie-Split_Erste_und_Weitere-Seiten-gemischt/train/ErsteSeiten/05.12.2022 - vn  668576245 - bauder robert ... - zur bearbeitung an vm(p015372975)_page_0.png"

In [ ]:
image = Image.open(path_to_img)
print(type(image))
image = image.convert("RGB")
image

In [ ]:
labels = dataset['train'].features['label']
labels = labels.names
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
id2label

In [ ]:
from collections import Counter

# Get the label column for the train and test datasets
train_labels = dataset['train']['label']
test_labels = dataset['test']['label']

# Count the number of instances for each label
train_label_counts = Counter(train_labels)
test_label_counts = Counter(test_labels)

# Print the label counts
print("Train Label Counts:")
print(train_label_counts)
print("\nTest Label Counts:")
print(test_label_counts)

## Creating a sampled dataset with n samples from each class

In [542]:
# Convert 'label' data from 'train' subset of 'dataset' into a DataFrame
dataset_df = pd.DataFrame(dataset['train']['label'], columns=['label'])

# Group the DataFrame by the unique values in the 'label' column
grouped_dataset_df = dataset_df.groupby('label')

# Randomly sample 23 rows from each group in the DataFrame
sample_dataset_df = grouped_dataset_df.sample(n=23, random_state=42)

# Select examples from dataset according to sampled index
dataset_sample_train = dataset['train'].select(sample_dataset_df.index)
dataset_sample_train

Dataset({
    features: ['image', 'image_path', 'label'],
    num_rows: 46
})

In [543]:
# Same steps as above for training data
test_dataset_df = pd.DataFrame(dataset['test']['label'], columns=['label'])
grouped_test_dataset_df = test_dataset_df.groupby('label')
sample_test_dataset_df = grouped_test_dataset_df.sample(n=8, random_state=42)
dataset_sample_test = dataset['test'].select(sample_test_dataset_df.index)
dataset_sample_test

Dataset({
    features: ['image', 'image_path', 'label'],
    num_rows: 16
})

In [ ]:
from transformers import AutoTokenizer, LayoutLMv2ImageProcessor, LayoutLMv3ImageProcessor
image_processor = LayoutLMv2ImageProcessor(ocr_lang='deu')
#image_processor = LiLTImageProcessor()
tokenizer = AutoTokenizer.from_pretrained("ChrisPCz/lilt-roberta-base-wechsel-german")
#lm_tokenizer = AutoTokenizer.from_pretrained("nielsr/lilt-roberta-en-base")

# Application of OCR via custom process()-method

In [ ]:
from functools import partial
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

"""
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'label': ClassLabel(num_classes=len(labels), names=labels),
})
"""

# Define the features of the dataset
features = Features({
    'image': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'label': ClassLabel(num_classes=len(labels), names=labels),
})

# Function to normalize bounding boxes
def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

# Function to preprocess data
def preprocess_data(examples, normalize_boxes = False):
    
    images = examples['image']
    
    # Convert images to RGB format
    images = [image.convert('RGB') for image in images]
    
    # Apply image processing to batch of images
    batchFeature = image_processor(images)
    
    boxes = []
    word_lists = []
    for word_array, box_array in zip(batchFeature['words'], batchFeature['boxes']):
        
        bbox = []
        
        
        for word, box in zip(word_array, box_array):
            
            # Normalize bounding box coordinates if required
            #box = normalize_bbox(box, width, height) if normalize_boxes else box
            
            # Extend the bounding box for each word in the word_array
            n_word_tokens = len(tokenizer.tokenize(word))
            bbox.extend([box] * n_word_tokens)
            
        cls_box = sep_box = [0, 0, 0, 0]
        
        # Adds [cls_box] at the beginning and [sep_box] at the end
        bbox = [cls_box] + bbox + [sep_box] # Gives a sequence length of 514 = 1 ([cls_box]) + 512 (bbox) + 1 ([sep_box])
    
        # Encode the word_array using tokenizer with truncation and padding
        word_encoding = tokenizer(' '.join(word_array), truncation=True, max_length=512, padding="max_length")
        sequence_length = len(word_encoding.input_ids)
        
        # Trim the bbox to match the sequence length
        bbox = bbox[:sequence_length]
        
        boxes.append(bbox)
        word_lists.append(' '.join(word_array))
 
    # Encode the word_lists using tokenizer with truncation and padding
    batchEncoding = tokenizer(word_lists, truncation=True, max_length=512, padding="max_length")#, return_tensors="pt")
    
    # Get the sequence length from the input_ids tensor
    sequence_length = torch.tensor(batchEncoding["input_ids"]).shape[1]
  
    # Add additional fields to the batchEncoding dictionary
    batchEncoding['image'] = batchFeature['pixel_values']
    batchEncoding["label"] = examples['label']
    
    # Create a list where each element 'boxes_example' in the 'boxes' list is concatenated with a repeated list '[[0, 0, 0, 0]]',
    # ensuring it matches the desired 'sequence_length'.
    # The resulting list is assigned to the 'bbox' key in the 'batchEncoding' dictionary.
    batchEncoding["bbox"] = [boxes_example + [[0, 0, 0, 0]] * (sequence_length - len(boxes_example)) for boxes_example in boxes]
    
    # convert to PyTorch
    # batch = {k: torch.tensor(v, dtype=torch.int64) if isinstance(v[0], list) else v for k, v in batch.items()}
    #batchEncoding = {k: torch.tensor(v) for k, v in batchEncoding.items()}
    
    return batchEncoding

In [ ]:
#token_encodings

In [ ]:
column_names = dataset['train'].column_names
column_names

In [ ]:
#dataset['train'].select([0,1])

In [ ]:
# Get the column names of the 'train' subset
column_names = dataset['train'].column_names

# Print the column names
print(column_names)

# Apply the 'preprocess_data' function to the 'train' subset of 'dataset_sample_train' in a batched manner
# with a batch size of 1. The 'features' variable specifies the expected structure of the processed data.
# The 'remove_columns' argument removes the columns specified in 'column_names' from the resulting processed dataset.
# The processed dataset is assigned to the 'proc_dataset_train' variable.
proc_dataset_train = dataset_sample_train.map(preprocess_data, batched=True, batch_size=1, features=features, remove_columns=column_names)

In [ ]:
# Repeat steps from above for test data
proc_dataset_test = dataset_sample_test.map(preprocess_data, batched=True, batch_size=1, features=features, remove_columns=column_names)

In [ ]:
proc_dataset_train = proc_dataset_train.remove_columns(['image'])
proc_dataset_train = proc_dataset_train.rename_column('label','labels')

In [ ]:
proc_dataset_test = proc_dataset_test.remove_columns(['image'])
proc_dataset_test = proc_dataset_test.rename_column('label','labels')

In [ ]:
#proc_dataset_train['labels']

In [ ]:
#proc_dataset_test['labels']

In [ ]:
proc_dataset_train

In [ ]:
for word, box in list(zip(proc_dataset_train['input_ids'][0], proc_dataset_train['bbox'][0])):
    print(word,box)

Draw and check bounding boxes

# Drawing and Check of Bounding Boxes

The `draw_multi_boxes()` method is to compare bounding boxes of datasets with bounding boxes obtained through different methods, e.g. Tesseract and Azure

In [ ]:
def unnormalize_box(bbox, width, height):
    return [
        width * (bbox[0] / 1000),
        height * (bbox[1] / 1000),
        width * (bbox[2] / 1000),
        height * (bbox[3] / 1000),
    ]

def draw_boxes(image, boxes, unnormalize_boxes = True):

    width, height = image.size
    print(width,height)
    
    boxes = [unnormalize_box(box, width, height) if unnormalize_boxes else box for box in boxes]

    # draw predictions over the image
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for box in boxes:
        draw.rectangle(box, outline="red")
            
    return image


def draw_multi_boxes(image, boxes1, boxes2, unnormalize_boxes = True):

    width, height = image.size
    print(width,height)
    
    boxes1 = [unnormalize_box(box, width, height) if unnormalize_boxes else box for box in boxes1]
    boxes2 = [unnormalize_box(box, width, height) if unnormalize_boxes else box for box in boxes2]

    # draw predictions over the image
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for box in boxes1:
        draw.rectangle(box, outline="red")
        
    for box in boxes2:
        draw.rectangle(box, outline="blue")
            
    return image

In [ ]:
# Requires image column, which was deleted from dataset above!
idx = 0
draw_boxes(dataset_sample_train['image'][idx], proc_dataset_train['bbox'][idx], True)

In [ ]:
for word, box in list(zip(proc_dataset_train['input_ids'][0], proc_dataset_train['bbox'][0])):
    print(tokenizer.decode(word),box)

In [ ]:
#print(id2label[0])
# Convert the token IDs back into words and print respective png
print(tokenizer.decode(proc_dataset_train['input_ids'][0]))
dataset_sample_train['image'][0]

In [ ]:
proc_dataset_train.set_format(type="torch", device="mps")

In [ ]:
proc_dataset_test.set_format(type="torch", device="mps")

# Define Model

In [ ]:
label2id.keys()

In [ ]:
num_labels=len(list(label2id.keys()))
num_labels

In [ ]:
from transformers import LiltForSequenceClassification, AutoModelForSequenceClassification
import torch

# Set random seed for PyTorch
torch.manual_seed(42)

device = torch.device("mps")
model = AutoModelForSequenceClassification.from_pretrained("ChrisPCz/lilt-roberta-base-wechsel-german", num_labels=len(list(label2id.keys())))

model.to(device)

In [ ]:
#model

In [ ]:
#model.__dict__

# Train Model

# Using HuggingFace-Trainer

In [ ]:
import accelerate
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_predictions):
    labels = eval_predictions.label_ids
    preds = eval_predictions.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

In [ ]:
from transformers import TrainerCallback

class ModelOutputCallback(TrainerCallback):
    def __init__(self, trainer, eval_dataset):
        self.trainer = trainer
        self.eval_dataset = eval_dataset
    
    def on_epoch_end(self, args, state, control, **kwargs):
        # get a few examples from the evaluation dataset
        eval_examples = self.eval_dataset[:3]
        
        # convert the examples to a format suitable for the model
        #encoded_eval_examples = self.trainer.tokenizer(eval_examples["image"], padding=True, truncation=True, return_tensors="pt")
        
        # move the inputs to the device
        #device = self.trainer.args.device
        #encoded_eval_examples = {k: v.to(device) for k, v in encoded_eval_examples.items()}
   

        eval_examples.remove_columns(['image','token_type_ids'])
        # get the model output
        model_output = self.trainer.model(**eval_examples)
        predictions = model_output.logits.argmax(-1)
        
        # print the model output
        print("Example inputs:")
        print(eval_examples)
        print("Predictions:")
        print(predictions)


In [47]:
# define the training arguments
output_dir = '/Users/christopherczaban/Local_Project_Files/LiLT_Model/LiLT_Roberta-base-german-wechsel_pma_classif_mixed_first_pages_and_further_pages'

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=0.0001,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=12,
    #weight_decay=0.01,
    push_to_hub=False,
    logging_dir=output_dir + "/" + "train_logging",
    logging_steps=2,
    #save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    use_mps_device=True
    #greater_is_better=True,
)

In [ ]:
shuffled_proc_dataset_train = proc_dataset_train.shuffle(seed=41)
shuffled_proc_dataset_test = proc_dataset_test.shuffle(seed=41)

In [ ]:
shuffled_proc_dataset_train.set_format(type="torch", device="mps")
shuffled_proc_dataset_test.set_format(type="torch", device="mps")

In [ ]:
print(shuffled_proc_dataset_train)
print(shuffled_proc_dataset_test)

In [ ]:
ind = 1
word_list = []
for i,j in enumerate(shuffled_proc_dataset_train['input_ids'][ind][:]):
    #print(i,j)
    #print(i, tokenizer.decode(j.item()))
    word_list.append(tokenizer.decode(j.item()))

word_list_from_ger_ocr = " ".join(word_list)

print("index:", ind)
print("label:", shuffled_proc_dataset_train['labels'][ind])
print(word_list_from_ger_ocr)
#dataset_sample_train['image'][ind]

In [ ]:
import platform
platform.platform()

In [ ]:
# create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=shuffled_proc_dataset_train,
    eval_dataset=shuffled_proc_dataset_test,
    #tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]#, ModelOutputCallback(trainer, shuffled_encoded_eval_dataset)]
)

In [49]:
# train the model
trainer.train()

/Users/christopherczaban/.envs/LiLT_m1/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.309800,0.994684,0.562500,0.533333,1.000000,0.695652
2,1.290300,0.244515,0.875000,0.800000,1.000000,0.888889
3,0.001900,0.335772,0.937500,0.888889,1.000000,0.941176
4,0.000500,0.100138,0.937500,1.000000,0.875000,0.933333
5,0.000200,0.471453,0.937500,0.888889,1.000000,0.941176
6,0.000200,0.521768,0.937500,0.888889,1.000000,0.941176
7,0.000100,0.621991,0.875000,0.800000,1.000000,0.888889
8,0.000100,0.649126,0.875000,0.800000,1.000000,0.888889
9,0.000100,0.623752,0.875000,0.800000,1.000000,0.888889
10,0.000100,0.599886,0.875000,0.800000,1.000000,0.888889


TrainOutput(global_step=144, training_loss=0.1289556363281008, metrics={'train_runtime': 1410.8227, 'train_samples_per_second': 0.391, 'train_steps_per_second': 0.102, 'total_flos': 154540832440320.0, 'train_loss': 0.1289556363281008, 'epoch': 12.0})

# Create Feature Enriched Dataset

In [554]:
# Define a function that computes an embedding vector for an examples
def create_compute_embedding_vector_function(model):
    # Define the closure
    def compute_embedding_vector(examples):

        # Convert each inner list to a tensor
        input_ids_tensors = [ids.clone().detach() for ids in examples['input_ids']]
        # Stack the tensors
        input_ids_stacked = torch.stack(input_ids_tensors)

        # Repeat for attention masks
        attention_mask_tensors = [mask.clone().detach() for mask in examples['attention_mask']]
        attention_mask_stacked = torch.stack(attention_mask_tensors)

        # Repeat for bboxes
        bbox_tensors = [bbox.clone().detach() for bbox in examples['bbox']]
        bbox_tensors_stacked = torch.stack(bbox_tensors)
        
        # Pack all inputs into a dictionary
        inputs = {
            'input_ids': input_ids_stacked,
            'attention_mask': attention_mask_stacked,
            'bbox': bbox_tensors_stacked,
            'return_dict': True,
            'output_hidden_states': True,
        }
        
        device = torch.device('mps')  # Use 'cpu' for CPU

        # Move all tensor items in the inputs dictionary to the device
        inputs = {name: tensor.to(device) if torch.is_tensor(tensor) else tensor for name, tensor in inputs.items()}

        # Call the model with the unpacked inputs
        with torch.no_grad():
            outputs = model(**inputs)

        # Compute prediction
        logits = outputs.logits
        normalized_logits = F.softmax(logits, dim=-1)

        # Get the index of the maximum logit for each example
        predicted_class_indices = logits.argmax(-1)

        # Convert logits and predicted_class_indices to lists
        logits_list = logits.tolist()
        normalized_logits_list = normalized_logits.tolist()
        predicted_class_indices_list = predicted_class_indices.tolist()

        # Now you can get the predicted class for each example
        predicted_classes = [id2label[idx] for idx in predicted_class_indices_list]
        
        #last_hidden_states = outputs.hidden_states[-1]#.size()#[-1][0]#.shape#[1][0]
        #CLS_token_embedding = last_hidden_states[:][0][:]
        
        # Assuming you have the tensor `last_hidden_states` with shape [batch_size, sequence_length, hidden_size]
        last_hidden_states = outputs.hidden_states[-1]
        batch_size = last_hidden_states.size(0)
        sequence_length = last_hidden_states.size(1)
        hidden_size = last_hidden_states.size(2)

        # Reshape the tensor to have shape [batch_size * sequence_length, hidden_size]
        reshaped_hidden_states = last_hidden_states.view(batch_size * sequence_length, hidden_size)

        # Extract embeddings from the first entry of each example
        embeddings = reshaped_hidden_states[torch.arange(0, batch_size * sequence_length, sequence_length), :]

        # Unbind the embeddings along the first dimension
        embeddings_unbound = embeddings.unbind(0)

        # Convert each tensor to a list
        CLS_token_embeddings = [embedding.tolist() for embedding in embeddings_unbound]
                 
        return CLS_token_embeddings, predicted_classes, normalized_logits_list
    
        # Return the closure
    return compute_embedding_vector

# Create the closure
compute_embedding_vector = create_compute_embedding_vector_function(trainer.model)

# Define a function that adds an embedding vector to an examples
def add_features(examples):
    
    # Compute the embedding vector for the examples
    embedding_vector, predicted_classes, normalized_logits_list = compute_embedding_vector(examples)
    # Add the embedding vector as a new feature
    examples['embedding'] = embedding_vector
    examples['predicted_classes'] = predicted_classes
    examples['norm_logits'] = normalized_logits_list
    return examples

In [653]:
from datasets import concatenate_datasets

def create_feature_enriched_dataset(dataset, processed_dataset):
    feature_enriched_dataset = processed_dataset.map(add_features, batched=True, batch_size=4)
    feature_enriched_dataset = concatenate_datasets([feature_enriched_dataset, dataset], axis=1)
    
    gt_classes = [id2label[idx] for idx in feature_enriched_dataset['labels']]
    feature_enriched_dataset = feature_enriched_dataset.add_column("gt_classes",gt_classes)
    
    #TODO: Think about adding text for Phoenix analysis
    #images = dataset['image']
    # Convert images to RGB format
    #images = [image.convert('RGB') for image in images]
    # Apply image processing to batch of images
    #batchFeature = image_processor(images)
    
    feature_enriched_dataset = feature_enriched_dataset.remove_columns(['labels','input_ids', 'attention_mask', 'bbox'])
    
    
    return feature_enriched_dataset

In [ ]:
images = dataset['image']
# Convert images to RGB format
images = [image.convert('RGB') for image in images]
# Apply image processing to batch of images
batchFeature = image_processor(images)
for word_array in batchFeature['words']
' '.join(word_array)
batchEncoding = tokenizer(word_lists, truncation=True, max_length=512, padding="max_length")

In [591]:
proc_dataset_test

tensor([[    0,    47,  6457,  ...,     1,     1,     1],
        [    0,    58, 22044,  ...,     1,     1,     1],
        [    0, 10413,  1397,  ...,     1,     1,     1],
        ...,
        [    0, 47020,   516,  ...,     1,     1,     1],
        [    0, 32833,   448,  ...,     1,     1,     1],
        [    0,    47,  6457,  ...,   395, 43818,     2]], device='mps:0')

In [606]:
# Create a new dataset with the added feature
#feature_enriched_test_dataset = proc_dataset_test.map(add_features, batched=True, batch_size=4)

In [618]:
feature_enriched_test_dataset = create_feature_enriched_dataset(dataset_sample_test, proc_dataset_test)
feature_enriched_test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'bbox', 'embedding', 'predicted_classes', 'norm_logits', 'image', 'image_path', 'label', 'gt_classes'],
    num_rows: 16
})

In [619]:
feature_enriched_train_dataset = create_feature_enriched_dataset(dataset_sample_train, proc_dataset_train)
feature_enriched_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'bbox', 'embedding', 'predicted_classes', 'norm_logits', 'image', 'image_path', 'label', 'gt_classes'],
    num_rows: 46
})

In [704]:
for gt, pred in zip(feature_enriched_test_dataset['gt_classes'],feature_enriched_test_dataset['predicted_classes']):
    print(gt, pred)

ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten WeitereSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten


In [703]:
for gt, pred in zip(feature_enriched_train_dataset['gt_classes'],feature_enriched_train_dataset['predicted_classes']):
    print(gt, pred)

ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten WeitereSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
ErsteSeiten ErsteSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeiten
WeitereSeiten WeitereSeite

In [572]:
from datasets import concatenate_datasets
feature_enriched_test_dataset = concatenate_datasets([feature_enriched_test_dataset, dataset_sample_test], axis=1)

In [624]:
feature_enriched_test_dataset['image']

[<PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2480x3509>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2480x3509>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2480x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2481x3508>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=2480x3509>,
 <PIL.PngImagePlugin.PngImageFile image 

In [659]:
feature_enriched_test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'bbox', 'embedding', 'predicted_classes', 'norm_logits', 'image', 'image_path', 'label', 'gt_classes'],
    num_rows: 16
})

In [691]:
feature_enriched_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'bbox', 'embedding', 'predicted_classes', 'norm_logits', 'image', 'image_path', 'label', 'gt_classes'],
    num_rows: 46
})

In [692]:
feature_enriched_test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'bbox', 'embedding', 'predicted_classes', 'norm_logits', 'image', 'image_path', 'label', 'gt_classes'],
    num_rows: 16
})

In [693]:
feature_enriched_train_df = pd.DataFrame(feature_enriched_train_dataset)
feature_enriched_test_df = pd.DataFrame(feature_enriched_test_dataset)
feature_enriched_train_df.drop(columns=['input_ids', 'attention_mask', 'bbox','image','label'], inplace=True)
feature_enriched_test_df.drop(columns=['input_ids', 'attention_mask', 'bbox','image','label'], inplace=True)
feature_enriched_train_df['prediction_score'] = feature_enriched_train_df['norm_logits'].apply(lambda x: x[0])
feature_enriched_test_df['prediction_score'] = feature_enriched_test_df['norm_logits'].apply(lambda x: x[0])

old_part = '/Users/christopherczaban/Local_Project_Files/Kundenprojekte_Code_lokal/Code_pma/Dokumente_Kategorisierung/Dokumente/PoV-Kategorie-Split_Erste_und_Weitere-Seiten-gemischt/'
new_part = 'http://localhost:8000/'

feature_enriched_train_df['image_path'] = feature_enriched_train_df['image_path'].str.replace(old_part, new_part)
feature_enriched_test_df['image_path'] = feature_enriched_test_df['image_path'].str.replace(old_part, new_part)

In [665]:
print(feature_enriched_train_df.columns)
print(feature_enriched_test_df.columns)

Index(['embedding', 'predicted_classes', 'norm_logits', 'image_path',
       'gt_classes', 'prediction_score'],
      dtype='object')
Index(['embedding', 'predicted_classes', 'norm_logits', 'image_path',
       'gt_classes', 'prediction_score'],
      dtype='object')


In [718]:
feature_enriched_test_df['image_path'][0]#.head()

'http://localhost:8000/test/ErsteSeiten/12.07.2019 - vn  786979637 - bacinski anja ... - zur bearbeitung an vm(p011253367)_page_0.png'

# Visualize and Analyze with Phoenix

In [708]:
import phoenix as px

In [709]:
schema = px.Schema(
    prediction_score_column_name="prediction_score",
    prediction_label_column_name="predicted_classes",
    actual_label_column_name="gt_classes",
    embedding_feature_column_names={
        "document_embedding": px.EmbeddingColumnNames(
            vector_column_name="embedding",
            link_to_data_column_name="image_path"
        ),
    },
    excluded_column_names=[
        "norm_logits",
    ],
)

In [710]:
print(feature_enriched_test_df['prediction_score'])
#print(feature_enriched_train_df['prediction_score'])

0     0.999468
1     0.999352
2     0.999476
3     0.999408
4     0.999476
5     0.004691
6     0.999357
7     0.999324
8     0.000460
9     0.000453
10    0.000538
11    0.000615
12    0.001982
13    0.000576
14    0.000957
15    0.000531
Name: prediction_score, dtype: float64


In [711]:
test_ds = px.Dataset(dataframe=feature_enriched_test_df, schema=schema, name="test")
train_ds = px.Dataset(dataframe=feature_enriched_train_df, schema=schema, name="training")

converting items in column `embedding` to numpy.ndarray, because they have the following type: list
converting items in column `embedding` to numpy.ndarray, because they have the following type: list


In [714]:
train_ds.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 46 entries, 2023-07-12 00:15:55.942106+00:00 to 2023-07-12 00:15:55.942106+00:00
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   embedding          46 non-null     object             
 1   predicted_classes  46 non-null     object             
 2   image_path         46 non-null     object             
 3   gt_classes         46 non-null     object             
 4   prediction_score   46 non-null     float64            
 5   prediction_id      46 non-null     object             
 6   timestamp          46 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), object(5)
memory usage: 2.9+ KB


In [713]:
test_ds.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16 entries, 2023-07-12 00:15:55.912861+00:00 to 2023-07-12 00:15:55.912861+00:00
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   embedding          16 non-null     object             
 1   predicted_classes  16 non-null     object             
 2   image_path         16 non-null     object             
 3   gt_classes         16 non-null     object             
 4   prediction_score   16 non-null     float64            
 5   prediction_id      16 non-null     object             
 6   timestamp          16 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), object(5)
memory usage: 1.0+ KB


In [715]:
session = px.launch_app(primary=test_ds, reference=train_ds)

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:57734/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [717]:
#px.active_session().view()

# Custom Image Processor

In [375]:
from typing import Dict, Optional, Union

import numpy as np

from transformers.utils import is_vision_available
from transformers.utils.generic import TensorType

from transformers.image_processing_utils import BaseImageProcessor, BatchFeature, get_size_dict
from transformers.image_transforms import resize, to_channel_dimension_format, to_pil_image
from transformers.image_utils import (
    ChannelDimension,
    ImageInput,
    PILImageResampling,
    infer_channel_dimension_format,
    is_batched,
    to_numpy_array,
    valid_images,
)

from transformers.utils import logging


if is_vision_available():
    import PIL


logger = logging.get_logger(__name__)


def normalize_box(box, width, height):
    return [
        int(1000 * (box[0] / width)),
        int(1000 * (box[1] / height)),
        int(1000 * (box[2] / width)),
        int(1000 * (box[3] / height)),
    ]


def apply_az_ocr(image: PIL.PngImagePlugin.PngImageFile):

    # apply OCR
    words, boxes = apply_az_image_to_data(image)
    
    pil_image = to_pil_image(image)
    image_width, image_height = pil_image.size

    # finally, normalize the bounding boxes
    normalized_boxes = []
    for box in boxes:
        normalized_boxes.append(normalize_box(box, image_width, image_height))

    assert len(words) == len(normalized_boxes), "Not as many words as there are bounding boxes"

    return words, normalized_boxes


def flip_channel_order(image: np.ndarray, data_format: Optional[ChannelDimension] = None) -> np.ndarray:
    input_data_format = infer_channel_dimension_format(image)
    if input_data_format == ChannelDimension.LAST:
        image = image[..., ::-1]
    elif input_data_format == ChannelDimension.FIRST:
        image = image[:, ::-1, ...]
    else:
        raise ValueError(f"Unsupported channel dimension: {input_data_format}")

    if data_format is not None:
        image = to_channel_dimension_format(image, data_format)
    return image


class LiLTImageProcessor(BaseImageProcessor):
    r"""
    Constructs a LiLT image processor.
    Args:
        do_resize (`bool`, *optional*, defaults to `True`):
            Whether to resize the image's (height, width) dimensions to `(size["height"], size["width"])`. Can be
            overridden by `do_resize` in `preprocess`.
        size (`Dict[str, int]` *optional*, defaults to `{"height": 224, "width": 224}`):
            Size of the image after resizing. Can be overridden by `size` in `preprocess`.
        resample (`PILImageResampling`, *optional*, defaults to `PILImageResampling.BILINEAR`):
            Resampling filter to use if resizing the image. Can be overridden by the `resample` parameter in the
            `preprocess` method.
        apply_ocr (`bool`, *optional*, defaults to `True`):
            Whether to apply the Tesseract OCR engine to get words + normalized bounding boxes. Can be overridden by
            `apply_ocr` in `preprocess`.
    """

    model_input_names = ["pixel_values"]

    def __init__(
        self,
        do_resize: bool = True,
        size: Dict[str, int] = None,
        resample: PILImageResampling = PILImageResampling.BILINEAR,
        apply_ocr: bool = True,
        **kwargs
    ) -> None:
        super().__init__(**kwargs)
        size = size if size is not None else {"height": 224, "width": 224}
        size = get_size_dict(size)

        self.do_resize = do_resize
        self.size = size
        self.resample = resample
        self.apply_ocr = apply_ocr

    def resize(
        self,
        image: np.ndarray,
        size: Dict[str, int],
        resample: PILImageResampling = PILImageResampling.BILINEAR,
        data_format: Optional[Union[str, ChannelDimension]] = None,
        **kwargs
    ) -> np.ndarray:
        """
        Resize an image to `(size["height"], size["width"])`.
        Args:
            image (`np.ndarray`):
                Image to resize.
            size (`Dict[str, int]`):
                Size of the output image.
            resample (`PILImageResampling`, *optional*, defaults to `PILImageResampling.BILINEAR`):
                Resampling filter to use when resizing the image.
            data_format (`str` or `ChannelDimension`, *optional*):
                The channel dimension format of the image. If not provided, it will be the same as the input image.
        """
        size = get_size_dict(size)
        if "height" not in size or "width" not in size:
            raise ValueError(f"The size dictionary must contain the keys 'height' and 'width'. Got {size.keys()}")
        output_size = (size["height"], size["width"])
        return resize(image, size=output_size, resample=resample, data_format=data_format, **kwargs)

    def preprocess(
        self,
        images: ImageInput,
        do_resize: bool = None,
        size: Dict[str, int] = None,
        resample: PILImageResampling = None,
        apply_ocr: bool = None,
        return_tensors: Optional[Union[str, TensorType]] = None,
        data_format: ChannelDimension = ChannelDimension.FIRST,
        **kwargs,
    ) -> PIL.Image.Image:
        """
        Preprocess an image or batch of images.
        Args:
            images (`ImageInput`):
                Image to preprocess.
            do_resize (`bool`, *optional*, defaults to `self.do_resize`):
                Whether to resize the image.
            size (`Dict[str, int]`, *optional*, defaults to `self.size`):
                Desired size of the output image after resizing.
            resample (`PILImageResampling`, *optional*, defaults to `self.resample`):
                Resampling filter to use if resizing the image. This can be one of the enum `PIL.Image` resampling
                filter. Only has an effect if `do_resize` is set to `True`.
            apply_ocr (`bool`, *optional*, defaults to `self.apply_ocr`):
                Whether to apply the Azure OCR engine to get words + bounding boxes.
            return_tensors (`str` or `TensorType`, *optional*):
                The type of tensors to return. Can be one of:
                    - Unset: Return a list of `np.ndarray`.
                    - `TensorType.TENSORFLOW` or `'tf'`: Return a batch of type `tf.Tensor`.
                    - `TensorType.PYTORCH` or `'pt'`: Return a batch of type `torch.Tensor`.
                    - `TensorType.NUMPY` or `'np'`: Return a batch of type `np.ndarray`.
                    - `TensorType.JAX` or `'jax'`: Return a batch of type `jax.numpy.ndarray`.
            data_format (`ChannelDimension` or `str`, *optional*, defaults to `ChannelDimension.FIRST`):
                The channel dimension format for the output image. Can be one of:
                    - `ChannelDimension.FIRST`: image in (num_channels, height, width) format.
                    - `ChannelDimension.LAST`: image in (height, width, num_channels) format.
        """
        do_resize = do_resize if do_resize is not None else self.do_resize
        size = size if size is not None else self.size
        size = get_size_dict(size)
        resample = resample if resample is not None else self.resample
        apply_ocr = apply_ocr if apply_ocr is not None else self.apply_ocr

        if not is_batched(images):
            images = [images]

        if not valid_images(images):
            raise ValueError(
                "Invalid image type. Must be of type PIL.Image.Image, numpy.ndarray, "
                "torch.Tensor, tf.Tensor or jax.ndarray."
            )

        if do_resize and size is None:
            raise ValueError("Size must be specified if do_resize is True.")

       

        if apply_ocr:
            words_batch = []
            boxes_batch = []
            for image in images:
                words, boxes = apply_az_ocr(image)
                words_batch.append(words)
                boxes_batch.append(boxes)
                
        # After azure ocr all transformations expect numpy arrays.
        images = [to_numpy_array(image) for image in images]

        if do_resize:
            images = [self.resize(image=image, size=size, resample=resample) for image in images]

        # flip color channels from RGB to BGR (as Detectron2 requires this)
        images = [flip_channel_order(image) for image in images]
        images = [to_channel_dimension_format(image, data_format) for image in images]

        data = BatchFeature(data={"pixel_values": images}, tensor_type=return_tensors)

        if apply_ocr:
            data["words"] = words_batch
            data["boxes"] = boxes_batch
        return data

# Azure Functions

In [376]:
def apply_az_image_to_data(image):
    
    from io import BytesIO
    
    DocumentAnalysisClient = az_auth()
    #poller = az_create_poller(path_to_sample_documents, DocumentAnalysisClient)


    with BytesIO() as output:
        image.save(output, format='PNG')
        image_bytes = output.getvalue()


    poller = DocumentAnalysisClient.begin_analyze_document(
            "prebuilt-document", document=image_bytes
        )

    pages = az_get_pages_results_from_poller(poller)

    document_word_list = []
    document_bbox_list = []

    for page in pages:
        page_word_list, page_bbox_list = get_az_word_and_bboxes_from_page(page)
        document_word_list.extend(page_word_list)
        document_bbox_list.extend(page_bbox_list)

    return document_word_list, document_bbox_list
    

In [377]:
def az_auth():
    
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.formrecognizer import DocumentAnalysisClient

    endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"] = "https://formulards.cognitiveservices.azure.com/"
    key = os.environ["AZURE_FORM_RECOGNIZER_KEY"] = "b98c6924f8cb47e7aa8ff058194adb32"

    return DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )


def az_create_poller(path_to_document, document_analysis_client):

    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document("prebuilt-read", document=f)

    return poller
        

def az_get_pages_results_from_poller(poller):
    
    return poller.result().pages
    

# Use function in loop through pages
def get_az_word_and_bboxes_from_page(page):

    lines = get_lines_from_page(page)
    word_list = []
    bbox_list = []
    
    for line in lines:
        bboxes, words = get_words_and_bboxes_from_line(line)
        if bboxes is not None:
            word_list.extend(words)
            bbox_list.extend(bboxes)
        
        
    return word_list, bbox_list


def get_lines_from_page(page):
    return page.lines


def get_words_and_bboxes_from_line(line):
    
    polygon = get_polygon_from_line(line)
    hf_lilt_bbox = get_az_hf_lilt_bbox_from_polygon(polygon)
    
    if hf_lilt_bbox is not None:
        words = get_words_from_line(line)
        bbox_list = len(words) * [hf_lilt_bbox]
        
        word_list = []
        [word_list.append(word.content) for word in words]
        
        return bbox_list, word_list
    else:
        return None, None


def get_words_from_line(line):
    return line.get_words()

def get_polygon_from_line(line):
    if not line.polygon:
        return None
    return line.polygon

def get_az_hf_lilt_bbox_from_polygon(polygon):
    
    if polygon is None:
        return None
    
    x_bbox = get_x_bbox(polygon)
    x1l = x_bbox[0]
    x2r = x_bbox[1]
    x3r = x_bbox[2]
    x4l = x_bbox[3]

    y_bbox = get_y_bbox(polygon)
    y1b = y_bbox[0]
    y2b = y_bbox[1]
    y3t = y_bbox[2]
    y4t = y_bbox[3]

    # h_position_embeddings = self.h_position_embeddings(bbox[:, :, 3] - bbox[:, :, 1])
    # w_position_embeddings = self.w_position_embeddings(bbox[:, :, 2] - bbox[:, :, 0])
    bw = x3r - x1l # x1l + bw = x1l + x3r - x1l
    bh = y3t - y2b # y3t + bh = y3t + y3t - y2b = 2*y3t - y2b
    # (left,top);(right,bottom)
    #return [x1l,y3t,x3r,y2b]
    
    if x1l < x1l+bw and y2b < y2b+bh:
        return [x1l,y2b,x1l+bw,y2b+bh]
    else:
        return None

def get_x_bbox(polygon):
    if not polygon:
        return []
    else:
        # left, right, right, left  
        return [p.x for p in polygon]
        
def get_y_bbox(polygon):
    if not polygon:
        return []
    else:
        # bottom, bottom, top, top
        return [p.y for p in polygon]


def get_az_page_width(page):
    return page.width

def get_az_page_height(page):
    return page.height

# The Necessity for Processing the Bounding Boxes "Manually"

As it can be seen in the following, application of the tokenizer on the dataset will remove the bounding boxes. This is because the tokenizer XLMRobertaFast has no facility for processing bounding boxes. On the other handside, the tokenization processes increases the number of tokens, because the words a split up. Consequently, the bounding boxes have to be multiplied to have equal length sequences of input-tokens and bounding boxes. Therefore we need a method or a class to do this for us, i.e. which creates features ('input_ids', 'attention_mask', 'bbox') , which we will see in the following.

image: <class 'numpy.ndarray'>
gray: <class 'numpy.ndarray'>
thresh: <class 'float'>
gray: <class 'numpy.ndarray'>
opening: <class 'numpy.ndarray'>
canny: <class 'numpy.ndarray'>

The images (pixel values) obtained through the image processor have a size of 224

In [ ]:
def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

class CustomDataset(Dataset):
  def __init__(self, dataset, tokenizer, normalize_boxes = False):
    self.dataset = dataset
    self.tokenizer = tokenizer
    self.normalize_boxes = normalize_boxes

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    # get item
    example = self.dataset[idx]
    image = example["images"]
    #labels = example["labels"]
    words = example["words"]
    boxes = example["boxes"]

    # prepare for the model
    width, height = image.size

    bbox = []
    for word, box in zip(words, boxes):
        box = normalize_bbox(box, width, height) if self.normalize_boxes else box
        #print(box)
        n_word_tokens = len(tokenizer.tokenize(word))
        bbox.extend([box] * n_word_tokens)
    #print(bbox)
    cls_box = sep_box = [0, 0, 0, 0]
    bbox = [cls_box] + bbox + [sep_box]

    encoding = self.tokenizer(" ".join(words), truncation=True, max_length=512, padding='max_length')
    sequence_length = len(encoding.input_ids)
    #print(sequence_length)
    # truncate boxes and labels based on length of input ids
    bbox = bbox[:sequence_length] 

    encoding["bbox"] = bbox
    encoding["labels"] = example["labels"]
    
    return encoding

In [ ]:
features_train = CustomDataset(proc_dataset_train, tokenizer)

In [ ]:
features_train[0].keys()

# Decode and Check `input_ids`

In [ ]:
for id, box in list(zip(features_train[0]['input_ids'],features_train[0]['bbox']))[0:20]:
    print(tokenizer.decode([id]),box)

# Define Pytorch Dataloader

In [ ]:
for feature in proc_dataset_train.features:
    print(feature)
    print(proc_dataset_train[feature].shape)

In [ ]:
import torch

dataloader_train = torch.utils.data.DataLoader(proc_dataset_train, batch_size=2)
batch_train = next(iter(dataloader_train))

In [ ]:
for k,v in batch_train.items():
  print(k, v.shape)

In [ ]:
import torch

dataloader_test = torch.utils.data.DataLoader(proc_dataset_test, batch_size=2)
batch_test = next(iter(dataloader_test))

In [ ]:
for k,v in batch_test.items():
  print(k, v.shape)

In [ ]:
We need a dataloader to load our features we obtained from our `CustomDataset`-Class and to provide them to the model during training. To accomplish this we need a data collator method serves as batch feature provider of the DataLoader object.  

In [ ]:
Check dimensions of feature batch:

In [ ]:
Checking decoded `input_ids`:

In [ ]:
tokenizer.decode(batch_test["input_ids"][0])

# Training with PyTorch

In [ ]:
len_data = len(dataset_sample_train)
len_data

In [ ]:
from transformers import AdamW
from tqdm.notebook import tqdm

#optimizer = AdamW(model.parameters(), lr=0.0001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#  lr=0.00001, batch_size=1, nr samples: 8 # <-- bad parameters
#  lr=0.0001, batch_size=1, nr samples: 8 # <-- bad parameters
#  lr=0.0005, batch_size=1, nr samples: 8 # <-- bad parameters
#  lr=0.001, batch_size=1, nr samples: 8 # <-- bad parameters
#  lr=0.0001, batch_size=2, nr samples: 8 # <-- good parameters
#  lr=0.0005, batch_size=2, nr samples: 8 # <-- bad parameters
#  lr=0.0001, batch_size=4, nr samples: 8 # <-- bad parameters

# Set random seed for optimizer
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(torch.get_default_dtype())
            torch.manual_seed(42)
            state[k].copy_(torch.randn_like(v))


global_step = 0
num_train_epochs = 20
# t_total seems to be irrelevant
# t_total = len(train_dataloader) * num_train_epochs # total number of training steps 

#put the model in training mode
model.train()
for epoch in range(num_train_epochs):
    print("Epoch:", epoch)
    running_loss = 0.0
    correct = 0
    for batch in tqdm(dataloader_train):
        # forward pass
        outputs = model(**batch)
        loss = outputs.loss

        running_loss += loss.item()
        predictions = outputs.logits.argmax(-1)
        correct += (predictions == batch['labels']).float().sum()
        print(predictions)
        print(correct)

        # backward pass to get the gradients 
        loss.backward()

        # update
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1

    print("Loss:", running_loss / batch["input_ids"].shape[0])
    accuracy = 100 * correct / len_data
    print("Training accuracy:", accuracy.item())


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set the initial learning rate
lr = 5e-5
num_train_epochs = 15
global_step = 0

optimizer = AdamW(model.parameters(), lr)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

total_steps = len(dataloader) * num_train_epochs
warmup_steps = int(total_steps * 0.1)
print("warmup_steps: ", warmup_steps)

scheduler_after_step = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

# put the model in training mode
model.train()

for epoch in range(num_train_epochs):
    print("Epoch:", epoch)
    running_loss = 0.0
    correct = 0
    for batch in tqdm(dataloader):
        # forward pass
        outputs = model(**batch)
        loss = outputs.loss

        running_loss += loss.item()
        predictions = outputs.logits.argmax(-1)
        correct += (predictions == batch['labels']).float().sum()

        # backward pass to get the gradients 
        loss.backward()

        # update
        if global_step < warmup_steps:
            print("step: ", step)
            lr_this_step = lr * (global_step / warmup_steps)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_this_step
                print("lr_this_step:", lr_this_step)

        else:
            scheduler_after_step.step()
            
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1

    avg_loss = running_loss / len(dataloader)
    accuracy = 100 * correct / len_data
    print(f"Epoch {epoch} - Loss: {avg_loss:.4f} - Training accuracy: {accuracy.item():.2f}%")
    
    # Evaluate on validation set and adjust learning rate
    """
    model.eval()
    with torch.no_grad():
        val_loss, val_accuracy = evaluate(model, val_dataloader)
    """ 
    scheduler.step(avg_loss)
    #scheduler.step(val_loss)
    #print(f"Validation Loss: {val_loss:.4f} - Validation accuracy: {val_accuracy:.2f}%")
    
    model.train()
